# 1. Initializations

## 1.1 General imports

In [ ]:
### data
import pandas as pd
import numpy as np

### machine Learning
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV, cross_validate
from sklearn.utils.class_weight import compute_sample_weight
from scipy.stats import chi2_contingency, randint, uniform
from skopt import BayesSearchCV 
from imblearn.metrics import classification_report_imbalanced
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

### graphical
import matplotlib.pyplot as plt
# for jupyter notebook management
%matplotlib inline
import seaborn as sns

## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 Load of the dataset

In [ ]:

df_raw = dfc.load_dataset_from_config('loan_data', sep=',', index_col=0)

if df_raw is not None and isinstance(df_raw, pd.DataFrame):
    dfc.log_general_info(df_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_raw))
    df = df_raw.copy()
    display(df.head())


In [ ]:
# a)
df.info()
display(df.describe())
display(df)

# 2. Full WalkThrough

In [ ]:
# Insérez votre code ici
# b)
missing_stats = df.isna().sum() / len(df)
missing_stats = (missing_stats[missing_stats > 0] * 100).round(2)
print("pourcentage de données manquantes pour les colonnes possédant des NaN")
print(missing_stats)

In [ ]:
print("colonne a supprimer:", missing_stats[missing_stats > 40].index)
df = df.drop(columns=missing_stats[missing_stats > 40].index)

In [ ]:
# Insérez votre code ici
# c) valeurs uniques
print("Nombre de valeurs unique pour emp_title:",df.emp_title.unique().size)
print(df.emp_title.value_counts()[df.emp_title.value_counts()>2000])
# il existe énormément de libellé d'emploi là où nous attendrions des titres catégorisés en centaines max
# la proportion de titre unique est donc d'environ 1/4 le volume de ligne (201348/808976)
# la donnée doit etre en saisie libre et il y a du coup des titres très similaires avec coquille 
# comme "Teacher" (le majoritaire) et "teacher" qu'on retrouve dans les plus représentés)
# cette source n'est donc pas fiable en terme de valeur explicative dans l'état et je décide de la 
# supprimer à cette étape de l'analyse
# d)
df = df.drop(columns=['emp_title'])

In [ ]:
# Insérez votre code ici
# e)
df = df.dropna(how='any', axis=0)
print("dimensions du dataframe après transformation",df.shape)
df.info()

In [ ]:
# Insérez votre code ici 
# f) proportion normalisée par rapport au nombre de valeur par statut
df['loan_status'].value_counts(normalize=True)
# g) 
# Situation A : certitude sur le remboursement : "Fully Paid"
# situation B : certitude sur le non remboursement : "Charged off"
# Situation C : il existe un risque variable de non remboursement : tous les autres situations
# Default pourrait être considéré comme un cas très vraissemblable de non remboursement mais la certitude n'est pas de 100%
# donc nous l'excluons et il consitute une partie marginale de toute façon

In [ ]:
# Insérez votre code ici 
# h) filtrage des colonnes dont le status n'est pas en situation A ou B
mask = (df.loan_status.isin(['Fully Paid','Charged Off']))
df = df.loc[mask]
print("dimensions du dataframe après transformation",df.shape)
# i) application de la transformation
df['current_loan_standing'] = df.loan_status.apply(lambda status: 0 if status=='Fully Paid' else 1)
target = df.current_loan_standing
print(f"proportion des valeurs de target:\n{target.value_counts(normalize=True)}\n")
print(f"Dimensions :{target.shape}\n")
# verification intermédiaires
print(target)
print(df.loan_status)
# il s'agit ici d'un problème de classification binaire avec une répartition de classe assymétrique (à considérer)

In [ ]:
# Insérez votre code ici
# m) 
df = df.drop('loan_status', axis=1)

In [ ]:
# Insérez votre code ici
# n)
print(f"proportions des grades:\n{df.grade.value_counts(normalize=True)}\n")
props_loan_by_grade = df.groupby("grade").current_loan_standing.value_counts(normalize=True)
print(f"proportions des prêts de classe 0 et 1 par grade:\n{props_loan_by_grade}")
props_grade_by_loan = df.groupby("current_loan_standing").grade.value_counts(normalize=True)
print(f"proportions des note grade pour les prêts de classe 0 et 1:\n{props_grade_by_loan}")
# on constate que pour le grade A, B, C, D les proportions sont dissymétrique et pour les classes E, F, G déjà plus
# homogènes
# On peut également déduire des observations qu'un prêt noté A correspond à un prêt qui a de forte chance d'être remboursé
# plus la note diminue et plus l'incertitude grandit

In [ ]:
# Insérez votre code ici 
# o)
display(df.grade)
df.grade = df.grade.replace(['A','B','C','D','E','F','G'], [6,5,4,3,2,1,0])
# verification
display(df.grade)

In [ ]:
# Insérez votre code ici
# p) 
# La stratégie pour maximiser le rendement serait d'apporter une aide à l'identification de la valeur prédite de remboursement
# pour les prêts avec grade à note faible (D,E,F,G donc note <=3) c'est là qu'il y a des taux d'emprunt elevés qui permettent 
# un bon retour sur investissement pour les prêteurs et c'est là que le besoin de prédire la classe négative sans se tromper
# (rappel de la classe 0 - prêt remboursé) est justement le plus criant (sans pour autant perdre en précision)
# on va donc se focaliser sur le f1-score (harmonique entre précision et rappel sur la classe positive)

# on pourrait probablement mettre un poids plus important aux prédictions des grades à notes faibles (avec ajustement manuel 
# du seuilde probabilité d'identification de la classe faible) et nous conserverions alors l'ensemble du dataset

# on pourrait également adopter une stratégie ou le data set d'entrainement suréchantillone les grades à note faible
# ou réciproquement sous-échantillonne les grades à note forte

# on pourrait aussi explorer un échantillonnage d'entrainement qui se focalise sur chaque grade ou groupes de grades
# et limiter notre outil à la prédiction fiable dans le contexte d'identification d'un grade

In [ ]:
# Insérez votre code ici 
# q) plusieurs axes pour éliminer des variables explicatives
# - les identifiants unique (amène facilement du sur-apprentissage)
# => "id" à supprimer 
# - les variables connue a posteriori (non disponible e production)
# => "total_pymnt" à supprimer

In [ ]:
# - On identifie également les variables explicatives quantitative introduisant potentiellement de la multicolinéarité 
# (ie. extrêment correlées entre elles avec test statistique à l'appui comme pearson avec |coeff| > 0.95 au moins
# pas d'exclusion à ce niveau (car il pourrait y avoir de la signifiance statistique pour des modèles multivariés
# on les garde en tête :
# => "loan_amnt" très correlé à "installment" : 0.95 (qui sont déduits par calculs incluant la variable "term")
# => "int_rate" très inversement correlé à "grade" : -0,96 (sans surprise d'apprendre que le grade conditionne le taux)
corr_matrix = df.select_dtypes(include='number').corr(method='pearson')
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", mask=np.triu(corr_matrix))
plt.title("Matrice de corrélation (Pearson)")
plt.show()
print("valeurs uniques installment\n",df.installment.unique())
print("valeurs uniques loan_amnt\n",df.loan_amnt.unique())

In [ ]:
# - On identifie enfin les variables explicatives qualitatives qui pourraient être problématiques
# le test du chi2 et la signifiance du V de Cramer permettent d'identifier leur signifiance et identifier
# de potentielles multicolinéarité par rapport à notre variable cible binaire (qualitative à deux valeurs)
# => aucune combinaison n'atteint une signifiance qui pourrait montrer une multicolinéarité donc on les conserve toutes
qual_cols = list(df.select_dtypes(exclude='number').columns)
target_col = 'current_loan_standing'
# parcours des colonnes qualitatives
for qual_col in qual_cols:
    print(f"\n== Traitement de la colonne [{qual_col}] ==\n")
    # Génération des colonnes dummies 
    qual_col_dummies = pd.get_dummies(df[qual_col], prefix=qual_col)
    print("Colonnes dummies générées :", list(qual_col_dummies.columns))
    # Pour chaque modalité de cette variable (dummy 0/1), tester sa signifiance avec la variable cible
    for col in qual_col_dummies:
        # Test du Chi-Deux
        cross_tab = pd.crosstab(df[target_col], qual_col_dummies[col])
        if cross_tab.shape[1] != 2:
            print(f"Modalité [{col}] ignorée (1 seule valeur présente)")
            continue
        stat, p, _, _ = chi2_contingency(cross_tab)
        # V de Cramer
        V_Cramer = np.sqrt(
            stat/cross_tab.values.sum())
        # On donne une pré-analyse des variables significatives (p-value < 5%) et V de Cramer supérieur à 0.1, et notamment
        # Faible : Valeur autour de 0.1
        # Moyenne : Valeur autour de 0.3
        # Elevée : Valeur autour et supérieure à 0.5
        # Lorsque la valeur du V de Cramer est très élevée (aux alentours de 0.8 et plus), on soupçonne généralement de la multicolinéarité.
        result = 'significative' if (p < 0.05) and (V_Cramer > 0.1) else 'NON signficative'  # type: ignore
        print(f"Variable [{col}] {result} Vs [{target_col}]: p-value[{p:.5f}], V_Cramer[{V_Cramer:.5f}]")

In [ ]:
# Insérez votre code ici 
# q) mise de côté des variables explicatives identifiées 
df = df.drop(columns=['id','total_pymnt'])
# On conserve pour le moment les deux quantitatives fortement corrélées
# df = df.drop(columns=['loan_amnt','int_rate'])

In [ ]:
# Insérez votre code ici
# r) Le pre-processing general (drop colonnes inutiles / filtrage NaN par seuil / sélection de features significatives) 
# a été fait dans les étapes ci-dessus on récupère juste les features en décidant de conserver les grades A, B, C dans 
# une première version d'un modèle de regression logistique simple
features = df.drop(columns=['current_loan_standing'])  # supprime la cible
# target est déjà alimenté avec la cible dans les étapes précédentes

## Stratégie 1 : Modèle Regression logistique avec BayesSearch

In [ ]:
# s) séparation train/test en conservant la répartition de la classe minoritaire positive afin de pouvoir entrainer 
# correctement nos modèle pour notre problématique métier
X_train, X_test, y_train, y_test = train_test_split(features, target, stratify=target, test_size=0.2, random_state=42)
# conservation des index des grade dans les features pour faire de l'analyse par suite
grade_test = features.loc[X_test.index, 'grade']

In [ ]:
# r_bis) pre-processing post train/split de regression logistique (fit sur train et propagation sur test):
# scaling des données quantitatives
s_scal_col = list(features.select_dtypes(include='number'))
s_scal = StandardScaler()
X_train[s_scal_col] = s_scal.fit_transform(X_train[s_scal_col])
X_test[s_scal_col] = s_scal.transform(X_test[s_scal_col])

# encodage en one hot des données qualitatives en droppant la première pour la multicolinéarité que cela induirait sinon
ohe_enc_col = list(features.select_dtypes(exclude='number').columns)
ohe_enc = OneHotEncoder(handle_unknown='ignore', sparse_output = False, drop='first')
# Appliquer OneHotEncoder
X_train_enc_cat = ohe_enc.fit_transform(X_train[ohe_enc_col])
X_test_enc_cat = ohe_enc.transform(X_test[ohe_enc_col])
# Ajout des colonnes encodées à un DataFrame car le resultat de enc.fit/transform est un ndarray sans index/colonnes
X_train_cat_df = pd.DataFrame(X_train_enc_cat, columns=ohe_enc.get_feature_names_out(ohe_enc_col), index=X_train.index)
X_test_cat_df = pd.DataFrame(X_test_enc_cat, columns=ohe_enc.get_feature_names_out(ohe_enc_col), index=X_test.index)  # type: ignore
# Suppression des colonnes catégoriques originales et ajout des colonnes encodées
X_train = pd.concat([X_train.drop(columns=ohe_enc_col), X_train_cat_df], axis=1)
X_test = pd.concat([X_test.drop(columns=ohe_enc_col), X_test_cat_df], axis=1)

In [ ]:
# vérifications
print("dimension X_train", X_train.shape)
print("dimension X_test", X_test.shape)
print("dimension y_train", y_train.shape)
print("dimension y_test", y_test.shape)

In [ ]:
# t) la métrique sera pour la classification binaire par regression logistique d'optimiser le recall sur la classe positive 
# (sans perdre en précision), si bien que le F1-Score devra être maximisé, nous pouvons donc envisager d'explorer la courbe 
# AUC-Precision-Recall pour trouver les seuils de déclenchement de probabilité qui maximise le F1-score par grade

In [ ]:
# u) Entrainement d'un modèle de Regression logistique linéaire (simple) avec grid search et exploration des résultats du modèle
# (cross validation et exploration de l'AUC-PR) en entrainement puis application et vérification avec l'échantillon de test
# on traite le déséquilibre de classe grace au paramètre class_weight qui va utiliser le ratio observé sur la variable cible
clf_lr = LogisticRegression(class_weight='balanced')
dico_param = {
    'solver': ['liblinear', 'lbfgs'],
    'C': (0.01, 1),
}
# recherche des hyperparamètre par rapport au f1_score
search_bs_clf_lr = BayesSearchCV(
    estimator=clf_lr, search_spaces=dico_param, scoring='f1',
    n_iter=10, cv=5, random_state=42
)
# Entrainement avec pondération par l'inverse du grade (les plus faibles (G, F, ...) étant les plus importants)
weights = 1 / (X_train['grade'] + 1)
search_bs_clf_lr.fit(X_train, y_train, sample_weight=weights)
# affichage des paramètres et récupération du meilleur estimateur trouvé 
print("Meilleurs paramètres de logistic regression trouvés",search_bs_clf_lr.best_params_)
best_clf_lr = search_bs_clf_lr.best_estimator_
# verification de la generalisation par cross validation sur les données d'entrainement
scoring = {
    'f1': 'f1',
    'precision': 'precision',
    'recall': 'recall'
}
cv_results = cross_validate(
    best_clf_lr, X_train, y_train,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring=scoring
)
print("Scores par cross validation stratifiée:\n")
for metric in scoring:
    scores = cv_results[f'test_{metric}']
    print(f"{metric} : mean={scores.mean():.3f}, std={scores.std():.3f}")
# Prediction sur les données de test
y_test_pred = best_clf_lr.predict(X_test)
# rapport de classification initial
cr = classification_report_imbalanced(y_test, y_test_pred)
print("\nRapport de classification initial :")
print(cr)
# matrice de confusion initiales
cm = confusion_matrix(y_test, y_test_pred)
display("Matrice de confusion initiale",pd.DataFrame(cm))
for num, label in zip(range(0,7),['G','F','E','D','C','B','A']):
    # rapport de classification par grade
    cr_grade = classification_report_imbalanced(y_test[grade_test==num], y_test_pred[grade_test==num])
    print(f"\nRapport de classification initial pour le grade {label} [{num}]:")
    print(cr_grade)
    # matrice de confusion initiale par grade
    cm_grade = confusion_matrix(y_test[grade_test==num],y_test_pred[grade_test==num])
    print(f"Matrice de confusion initiale pour le grade {label} [{num}]")
    display(pd.DataFrame(cm_grade))

# Recherche des meilleurs seuil de probabilité pour optimiser le F1-score par grade
y_test_probas = best_clf_lr.predict_proba(X_test)[:, 1]
thresholds_by_grade = {}
for grade_val in sorted(grade_test.unique()):
    mask = (grade_test == grade_val)
    y_true_g = y_test[mask]
    y_prob_g = y_test_probas[mask]
    prec, rec, thresholds = precision_recall_curve(y_true_g, y_prob_g, pos_label=1)
    f1_scores = 2 * (prec * rec) / (prec + rec + 1e-8)
    best_idx = f1_scores.argmax()
    best_threshold = thresholds[best_idx]
    auc_pr = auc(rec, prec)
    # calcul de l'AUC -PR (intégrale de la répartition precision/recall selon les seuils de probas)
    thresholds_by_grade[grade_val] = {
        'threshold': best_threshold,
        'auc_pr': auc_pr,
        'f1': f1_scores[best_idx],
        'precision': prec[best_idx],
        'recall': rec[best_idx]
    }
    print(f"Grade {grade_val} — Seuil optimal: {best_threshold:.4f}, AUC-PR: {auc_pr:.4f}, F1: {f1_scores[best_idx]:.4f}")

# Calcul des predictions avec seuils optimisés
y_test_preds_by_grade = []
for idx, (proba, grade_val) in enumerate(zip(y_test_probas, grade_test)):
    th = thresholds_by_grade[grade_val]['threshold']
    pred = int(proba >= th)
    y_test_preds_by_grade.append(pred)
y_test_pred_custom = np.array(y_test_preds_by_grade)
# rapport de classification avec les seuils optimisés
cr_opti = classification_report_imbalanced(y_test, y_test_pred_custom)
print("\nRapport de classification sur le seuil proba optimisé pour F1 :")
print(cr_opti)
# matrice de confusion avec les seuils optimisés (globale et par grade)
cm_opti = confusion_matrix(y_test, y_test_pred_custom)
display("Matrice de confusion sur le seuil optimisé pour F1",pd.DataFrame(cm))
for num, label in zip(range(0,7),['G','F','E','D','C','B','A']):
    # rapport de classification par grade
    cr_opti = classification_report_imbalanced(y_test[grade_test==num], y_test_pred_custom[grade_test==num])
    print(f"\nRapport de classification sur le seuil optimisé pour F1 pour le grade {label} [{num}]:")
    print(cr_opti)
    # matrice de confusion initiale par grade
    cm_opti_grade = confusion_matrix(y_test[grade_test==num],y_test_pred_custom[grade_test==num])
    print(f"Matrice de confusion sur le seuil optimisé pour F1 pour le grade {label} [{num}]")
    display(pd.DataFrame(cm_opti_grade))


In [ ]:
# Analyse globale des résultats:
# on constate que le modèle même s'il donne des résultat supérieurs au simple choix aléatoire, n'est pas encore satisfaisant 
# par rapport à notre problématique métier : la précision sacrifie beaucoup de recall et inversement
# par ailleurs avec le paramètre F1 optimisé sur la probabilité de déclenchement on atteint tout juste 0.45 pour la classe positive
# les matrices de confusion sur les grades D,E,F,G montrent que les recall/precision s'aggravent nettement 
# jusqu'à une spécificité quasi totale

# La répartition des proportions d'échantillons et du déséquilibre de classe ne permet pas au modèle de s'ajuster 
# correctement malgré le paramètre class_weight='balanced' et l'affectation de poids au grade lors de l'entrainement
# le modèle qui vise une relation linéaire entre beaucoup de variables explicatives peut être modifié pour utiliser
# un modèle sur chaque grade, non linéaire et plus tolérant aux variables explicatives qualitatives sans trandformation 
# XGBoostClassifier serait un bon candidat avec une grille de recherche optimisée (RandomSearch) 
# sur des données d'entrainement qu'on pourra sous échantillonner sur la classe minoritaire (ou sur échantilloner sur 
# la classe majoritaire)

## Stratégie 2 : Modèle XGBoost ciblé sur grade + RandomizedSearchCV et oversampling sur Grade "G"

In [ ]:
# s) séparation train/test sur le grade G
df_gr = df.loc[df.grade==0] # grade G
features_gr = df_gr.drop(columns=['current_loan_standing'])  # supprime la cible
target_gr = df_gr['current_loan_standing']
X_train_gr, X_test_gr, y_train_gr, y_test_gr = train_test_split(features_gr, target_gr, stratify=target_gr, test_size=0.2, random_state=42)

In [ ]:
# r_bis) pre-processing post train/split pour XGboost:
# oversampling de la classe minoritaire
ros_gr = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_gr, y_train_gr = ros_gr.fit_resample(X_train_gr, y_train_gr)

# transformation des variables catégorielles explicatives en category pour XGBoost (et SMOTETomek)
for col in df_gr.select_dtypes(include='object').columns:
    categories = df_gr[col].astype("category").cat.categories
    X_train_gr[col] = pd.Categorical(X_train_gr[col], categories=categories)
    X_test_gr[col] = pd.Categorical(X_test_gr[col], categories=categories)

print("Répartition :")
print(pd.Series(y_train_gr).value_counts(normalize=True).round(3))


In [ ]:
# vérifications
print("dimension X_train grade", X_train_gr.shape)
print("dimension X_test grade", X_test_gr.shape)
print("dimension y_train grade", y_train_gr.shape)
print("dimension y_test grade", y_test_gr.shape)

In [ ]:
# u) Entrainement d'un modèle de classification eXtrême Gardient BOOSTing (non linéaire) avec random search et exploration 
# des résultats du modèle (cross validation et exploration de l'AUC-PR) en entrainement puis application et vérification 
# avec l'échantillon de test
def xgb_model_pipeline(X_tr, y_tr, X_te, y_te):
    clf_xgb = XGBClassifier(
        enable_categorical=True,
    )
    dico_param = {
        'n_estimators': randint(50, 200),
        'learning_rate': uniform(0.01, 0.3),
        'max_depth': randint(3, 10),
        'min_child_weight': randint(1, 10), # tuning du surapprentissage
        'gamma': uniform(0, 5), # complexité/compréhension de l'arbre
        'reg_lambda': uniform(0, 5), # regularisation L2 (Ridge : robustesse au surapprentissage)
        'reg_alpha': uniform(0, 5), # régularisation L1 (Lasso : mise de côté des variables peu explicatives)
    }
    # recherche des hyperparamètres par rapport au f1_score en randomized search
    search_rs_clf_xgb = RandomizedSearchCV(
        estimator=clf_xgb, param_distributions=dico_param, scoring='f1',
        n_iter=30, cv=5, verbose=1, n_jobs=-1, random_state=42, 
    )
    # Entrainement en prenant en compte les poids des classes (NB : non nécessaire si over/under sampling 50/50)
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_tr)
    search_rs_clf_xgb.fit(X_tr, y_tr, sample_weight=sample_weights)
    # affichage des paramètres et récupération du meilleur estimateur trouvé 
    print("Meilleurs paramètres de XGBoost trouvés",search_rs_clf_xgb.best_params_)
    best_clf_xgb = search_rs_clf_xgb.best_estimator_
    # verification de la generalisation par cross validation
    scoring = {
        'f1': 'f1',
        'precision': 'precision',
        'recall': 'recall'
    }
    cv_results = cross_validate(
        best_clf_lr, X_train, y_train,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring=scoring
    )
    print("Scores par cross validation stratifiées:\n")
    for metric in scoring:
        scores = cv_results[f'test_{metric}']
        print(f"{metric} : mean={scores.mean():.3f}, std={scores.std():.3f}")
    # Prediction sur les données de test
    y_te_pred = best_clf_xgb.predict(X_te)
    # rapport de classification sur les données de test
    cr = classification_report_imbalanced(y_te, y_te_pred)
    print("\nRapport de classification :")
    print(cr)
    # matrice de confusion sur les données de test
    cm = confusion_matrix(y_te, y_te_pred)
    display("Matrice de confusion",pd.DataFrame(cm))

In [ ]:
xgb_model_pipeline(X_train_gr, y_train_gr, X_test_gr, y_test_gr)

# Analyse du résultat pour le grade G :
# on constate que le modèle optimisé pour le grade est plus homogène entre la classe positive et négative
# avec une moyenne géométrique passant de 0 à 0.57 (donc une bien meilleure distribution entre les deux classes)
# le F1-score est aussi meilleur sur le jeu de test que sur la moyenne des cross validation du jeu d'entrainement
# donc une bonne généralisation aux nouvelles données pour l'application à notre problématique métier
# d'identification sur les grades à fort rendement (taux d'emprunt élevé)

In [ ]:
# s) séparation train/test sur le grade F
df_gr = df.loc[df.grade==1] # grade F
features_gr = df_gr.drop(columns=['current_loan_standing'])  # supprime la cible
target_gr = df_gr['current_loan_standing']
X_train_gr, X_test_gr, y_train_gr, y_test_gr = train_test_split(features_gr, target_gr, stratify=target_gr, test_size=0.2, random_state=42)

In [ ]:
# r_bis) pre-processing post train/split pour XGboost:
# oversampling de la classe minoritaire
ros_gr = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_gr, y_train_gr = ros_gr.fit_resample(X_train_gr, y_train_gr)

# transformation des variables catégorielles explicatives en category pour XGBoost (et SMOTETomek)
for col in df_gr.select_dtypes(include='object').columns:
    categories = df_gr[col].astype("category").cat.categories
    X_train_gr[col] = pd.Categorical(X_train_gr[col], categories=categories)
    X_test_gr[col] = pd.Categorical(X_test_gr[col], categories=categories)

print("Répartition :")
print(pd.Series(y_train_gr).value_counts(normalize=True).round(3))


In [ ]:
# vérifications
print("dimension X_train grade", X_train_gr.shape)
print("dimension X_test grade", X_test_gr.shape)
print("dimension y_train grade", y_train_gr.shape)
print("dimension y_test grade", y_test_gr.shape)

In [ ]:
xgb_model_pipeline(X_train_gr, y_train_gr, X_test_gr, y_test_gr)

# Analyse du résultat pour le grade F :
# on constate que le modèle optimisé pour le grade est plus homogène entre la classe positive et négative
# moyenne géométrique passant de 0.03 à 0.58 (bien meilleure distribution entre les deux classes)
# le F1-score est la aussi meilleur sur le jeu de test que sur la moyenne des cross validation du jeu d'entrainement
# avec donc une bonne généralisation à de nouvelles données et là aussi pluys satisfaisant par rapport à la problématique 
# métier